In [1]:
import torch
from Feature import feature
from Utils import recall, recall2, recall2_batch
from torchvision import transforms
from torchvision import models
import torch.nn as nn
from Reader import ImageReader
from Utils import tra_transforms, eva_transforms
from color_lib import RGBmean, RGBstdv

In [2]:
Data = "HAR"
imgsize = 160
tra_transform = tra_transforms(imgsize, RGBmean[Data], RGBstdv[Data])
eva_transform = eva_transforms(imgsize, RGBmean[Data], RGBstdv[Data])

In [3]:
dsets_dict = torch.load("/notebooks/Image_Retrieval/EasyPositiveHardNegative-master/data_dict_emb_test.pth")
# dsets_dict = {p: ImageReader(data_dict[p], eva_transform) for p in phase}
model = models.resnet50(pretrained=True)
print('Setting model: resnet50')
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 64)
checkpoint = torch.load("/notebooks/Image_Retrieval/EasyPositiveHardNegative-master/_result/EPSHN/HAR_R50/G16_lr0.03/model_state_dict_R5050.pth")
model.load_state_dict(checkpoint)


/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Setting model: resnet50


<All keys matched successfully>

In [5]:
# model.load_state_dict(checkpoint)
model.eval()

In [6]:
dsets = ImageReader(dsets_dict['gallery'], tra_transform)
dsets1 = ImageReader(dsets_dict['query'], tra_transform)

In [29]:
class_names = dsets1.class_to_idx
(class_names)
inv_map = {v: k for k, v in class_names.items()}
inv_map

{0: 'calling',
 1: 'clapping',
 2: 'cycling',
 3: 'dancing',
 4: 'drinking',
 5: 'eating',
 6: 'fighting',
 7: 'hugging',
 8: 'laughing',
 9: 'listening_to_music',
 10: 'running',
 11: 'sitting',
 12: 'sleeping',
 13: 'texting',
 14: 'using_laptop'}

In [158]:
 get_class_name_from_id(dsets1.imgs[55][1],inv_map)

'eating'

In [33]:
def get_class_name_from_id(class_id_of_image,class_name_dict):
    class_name = class_name_dict[class_id_of_image]
    return class_name
class_name = get_class_name_from_id(0,inv_map)
class_name    

'calling'

In [35]:
acc = recall2(feature(dsets, model),
                      feature(dsets1, model), 
                      dsets.idx_to_class, 
                      dsets1.idx_to_class)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 48 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [35]:
Fvec_val = feature(dsets1, model)
Fvec_gal = feature(dsets, model)
imgLab_val = dsets1.idx_to_class
imgLab_gal = dsets.idx_to_class

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 48 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## mAP@k=1

In [268]:
N = len(imgLab_val)
imgLab_val = torch.LongTensor([imgLab_val[i] for i in range(len(imgLab_val))])
imgLab_gal = torch.LongTensor([imgLab_gal[i] for i in range(len(imgLab_gal))])

In [269]:
D = Fvec_val.mm(torch.t(Fvec_gal))
preds_1 = []
_,idx = D.max(1)
imgPre = imgLab_gal[idx]
preds_1.append(imgPre)
A = (imgPre==imgLab_val).float()
recall_1 = (torch.sum(A)/N).item()


In [270]:
recall_1

0.753333330154419

In [271]:
precision_1=[]
rank_1 = [0]*150
k=1
for i in range(150):
    relevant_count = 0
    precision_sum = 0
    for j in range(k):
        if((preds_1[j][i])==imgLab_val[i]):
            # print(preds1[i])
            if(rank_1[i]==0):
                rank_1[i] = (j+1)
            relevant_count += 1
            precision_sum += relevant_count / (j + 1)
            # print(precision_sum)
    ap = precision_sum / k
        # print(ap)
            
    precision_1.append(ap)

In [272]:
sum_precision_1 = sum(precision_1)

In [273]:
map_1 = sum_precision_1/N
map_1

0.7533333333333333

In [118]:
sum(rank_1)/150

0.7533333333333333

In [266]:
no_relevant_img_1=[]
for i in range(len(rank_1)):
    if rank_1[i] == 0:
        no_relevant_img_1.append(i)
(no_relevant_img_1)

[0,
 5,
 8,
 9,
 13,
 16,
 35,
 44,
 45,
 47,
 50,
 51,
 55,
 60,
 64,
 66,
 70,
 74,
 75,
 78,
 79,
 92,
 93,
 97,
 98,
 100,
 110,
 111,
 117,
 118,
 125,
 132,
 135,
 138,
 139,
 145,
 148]

In [275]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw, ImageFont

query_images = 150
columns = 3
rows = 2  # 1 query image + 50 retrieved images

# Adjust the size of your images as necessary
image_width = 128
image_height = 128

# Set up canvas size
canvas_width = 400
canvas_height =  400

for i in range(query_images):
    # Create a blank canvas
    canvas = Image.new('RGB', (canvas_width, canvas_height), (255, 255, 255))
    draw = ImageDraw.Draw(canvas)

    # Load and paste the query image
    query_image = dsets1.imgs[i][0]
    query_img_class = get_class_name_from_id(dsets1.imgs[i][1], inv_map)
    image = Image.open(query_image)
    image = image.resize((image_width, image_height))
    canvas.paste(image, (0, 0))
    draw.text((10, 10), f"Query:{query_img_class}", fill="white")

    # Load and paste the retrieved images
    candidates_idxx = []
    candidates_idxx.append(idx[i].tolist())
    
    for j, idxx in enumerate(candidates_idxx):
        image_path = dsets.imgs[idxx][0]
        image_class = get_class_name_from_id(dsets.imgs[idxx][1], inv_map)
        image = Image.open(image_path)
        image = image.resize((image_width, image_height))
        x_pos = (j % columns+2) * image_width
        y_pos = ((j // columns) + 1) * image_height  # +1 to shift down past the query image
        canvas.paste(image, (x_pos, y_pos))
        draw.text((x_pos + 10, y_pos + 10), f"Retrieved:{image_class}", fill="white")

    # Save the canvas as an image
    canvas.save(f'results_k_1/query_{i}_results.png')

In [61]:
D


tensor([[ 0.0236, -0.1163, -0.2755,  ...,  0.4113,  0.5511,  0.2319],
        [ 0.0830,  0.3949,  0.0047,  ...,  0.1088,  0.0916,  0.1578],
        [ 0.3001, -0.0565,  0.2835,  ..., -0.1077, -0.1218,  0.1212],
        ...,
        [ 0.2729,  0.4691,  0.2628,  ...,  0.6230,  0.3767,  0.6189],
        [ 0.1831,  0.1627,  0.4251,  ...,  0.3030,  0.1257,  0.2952],
        [ 0.1420,  0.3169,  0.1142,  ...,  0.5709,  0.4008,  0.4919]])

In [52]:
D.shape

torch.Size([150, 1000])

### mAP @ K=10

In [213]:
rank=[10]

In [214]:
N = len(imgLab_val)
imgLab_val = torch.LongTensor([imgLab_val[i] for i in range(len(imgLab_val))])
imgLab_gal = torch.LongTensor([imgLab_gal[i] for i in range(len(imgLab_gal))])

D = Fvec_val.mm(torch.t(Fvec_gal))
_,idx = D.topk(rank[-1])
acc_list = []
preds_10 = []
for r in rank:
    A = 0
    for i in range(r):
        imgPre = imgLab_gal[idx[:,i]]
        preds_10.append(imgPre)
        A += (imgPre==imgLab_val).float()
    acc_list.append((torch.sum((A>0).float())/N).item())
recall_10=torch.Tensor(acc_list)

In [215]:
recall_10

tensor([0.8933])

In [137]:
precision_10=[]
rank_10 = [0]*150
k=10
for i in range(N):
    relevant_count = 0
    precision_sum = 0
    for j in range(k):
        if((preds_10[j][i])==imgLab_val[i]):
            # print(preds1[i])
            if(rank_10[i]==0):
                rank_10[i] = (j+1)
            relevant_count += 1
            precision_sum += relevant_count / (j + 1)
            # print(precision_sum)
    ap = precision_sum / k
        # print(ap)
            
    precision_10.append(ap)
    

In [138]:
sum_precision_10 = sum(precision_10)

In [151]:
map_10 = sum_precision_10/N
map_10

0.7059148148148149

In [140]:
sum(rank_10)/N

1.22

In [141]:
no_relevant_img_10=[]
for i in range(len(rank_10)):
    if rank_10[i] == 0:
        no_relevant_img_10.append(i)
no_relevant_img_10

[5, 9, 16, 45, 50, 55, 70, 74, 75, 92, 93, 97, 125, 132, 139, 148]

Visualise or save retrieval results

In [255]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw, ImageFont

query_images = 150
columns = 3
rows = 11  # 1 query image + 50 retrieved images

# Adjust the size of your images as necessary
image_width = 128
image_height = 128

# Set up canvas size
canvas_width = 700
canvas_height =  700

for i in range(query_images):
    # Create a blank canvas
    canvas = Image.new('RGB', (canvas_width, canvas_height), (255, 255, 255))
    draw = ImageDraw.Draw(canvas)

    # Load and paste the query image
    query_image = dsets1.imgs[i][0]
    query_img_class = get_class_name_from_id(dsets1.imgs[i][1], inv_map)
    image = Image.open(query_image)
    image = image.resize((image_width, image_height))
    canvas.paste(image, (0, 0))
    draw.text((10, 10), f"Query:{query_img_class}", fill="white")

    # Load and paste the retrieved images
    candidates_idxx = idx[i].tolist()
    for j, idxx in enumerate(candidates_idxx):
        image_path = dsets.imgs[idxx][0]
        image_class = get_class_name_from_id(dsets.imgs[idxx][1], inv_map)
        image = Image.open(image_path)
        image = image.resize((image_width, image_height))
        x_pos = (j % columns+2) * image_width
        y_pos = ((j // columns) + 1) * image_height  # +1 to shift down past the query image
        canvas.paste(image, (x_pos, y_pos))
        draw.text((x_pos + 10, y_pos + 10), f"Retrieved:{image_class}", fill="white")

    # Save the canvas as an image
    canvas.save(f'results_k_10/query_{i}_results.png')

## mAP@k=50

In [256]:
rank=[50]

In [257]:
N = len(imgLab_val)
imgLab_val = torch.LongTensor([imgLab_val[i] for i in range(len(imgLab_val))])
imgLab_gal = torch.LongTensor([imgLab_gal[i] for i in range(len(imgLab_gal))])

D = Fvec_val.mm(torch.t(Fvec_gal))
_,idx = D.topk(rank[-1])
acc_list = []
preds_50 = []
for r in rank:
    A = 0
    for i in range(r):
        imgPre = imgLab_gal[idx[:,i]]
        preds_50.append(imgPre)
        A += (imgPre==imgLab_val).float()
    acc_list.append((torch.sum((A>0).float())/N).item())
recall_50=torch.Tensor(acc_list)

In [258]:
recall_50

tensor([0.9667])

In [259]:
precision_50=[]
rank_50 = [0]*150
k=50
for i in range(N):
    relevant_count = 0
    precision_sum = 0
    for j in range(k):
        if((preds_50[j][i])==imgLab_val[i]):
            # print(preds1[i])
            if(rank_50[i]==0):
                rank_50[i] = (j+1)
            relevant_count += 1
            precision_sum += relevant_count / (j + 1)
            # print(precision_sum)
    ap = precision_sum / k
        # print(ap)
            
    precision_50.append(ap)

In [260]:
sum_precision_50 = sum(precision_50)

In [261]:
map_50 = sum_precision_50/N
map_50

0.6097017321715196

In [262]:
sum(rank_50)/N

2.8733333333333335

In [263]:
no_relevant_img_50=[]
for i in range(len(rank_50)):
    if rank_50[i] == 0:
        no_relevant_img_50.append(i)
no_relevant_img_50

[5, 45, 55, 75, 93]

Visualise results for k=50

In [265]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw, ImageFont

query_images = 150
columns = 3
rows = 51  # 1 query image + 50 retrieved images

# Adjust the size of your images as necessary
image_width = 128
image_height = 128

# Set up canvas size
canvas_width = 700
canvas_height =  1000

for i in range(2):
    # Create a blank canvas
    canvas = Image.new('RGB', (canvas_width, canvas_height), (255, 255, 255))
    draw = ImageDraw.Draw(canvas)

    # Load and paste the query image
    query_image = dsets1.imgs[i][0]
    query_img_class = get_class_name_from_id(dsets1.imgs[i][1], inv_map)
    image = Image.open(query_image)
    image = image.resize((image_width, image_height))
    canvas.paste(image, (0, 0))
    draw.text((10, 10), f"Query:{query_img_class}", fill="white")

    # Load and paste the retrieved images
    candidates_idxx = idx[i].tolist()
    for j, idxx in enumerate(candidates_idxx):
        image_path = dsets.imgs[idxx][0]
        image_class = get_class_name_from_id(dsets.imgs[idxx][1], inv_map)
        image = Image.open(image_path)
        image = image.resize((image_width, image_height))
        x_pos = (j % columns+2) * image_width
        y_pos = ((j // columns) + 1) * image_height  # +1 to shift down past the query image
        canvas.paste(image, (x_pos, y_pos))
        draw.text((x_pos + 10, y_pos + 10), f"Retrieved:{image_class}", fill="white")

    # Save the canvas as an image
    canvas.save(f'results_k_50/query_{i}_results.png')

## Get mean and std

In [10]:
import os
import torchvision

In [4]:
os.listdir("/notebooks/Image_Retrieval/Hierarchy-image-retrieval-main/dataset/train_data")

['sleeping',
 'clapping',
 'laughing',
 'listening_to_music',
 'dancing',
 'calling',
 'sitting',
 'hugging',
 'using_laptop',
 'running',
 'eating',
 'texting',
 'cycling',
 'fighting',
 'drinking']

In [5]:
training_dataset_path = "/notebooks/Image_Retrieval/Hierarchy-image-retrieval-main/dataset/train_data"

In [8]:
training_transforms = transforms.Compose([transforms.Resize((160,160)),transforms.ToTensor()])

In [13]:
train_dataset = torchvision.datasets.ImageFolder(root=training_dataset_path,transform=training_transforms)

In [14]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=128,shuffle=False)

In [23]:
def get_mean_and_std(loader):
    mean = 0
    std = 0
    total_images_count = 0
    for images,_ in loader:
        images_count_in_a_batch = images.size(0)
        images = images.view(images_count_in_a_batch,images.size(1),-1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += images_count_in_a_batch
    
    mean /= total_images_count
    std /= total_images_count
    return mean,std
    

In [24]:
get_mean_and_std(train_loader)

(tensor([0.5717, 0.5368, 0.5057]), tensor([0.2396, 0.2372, 0.2406]))